<a href="https://colab.research.google.com/github/mariajosemuskusl/Integracion-de-Datos-y-Prospectiva/blob/main/2_Integraci%C3%B3n_Credibilidad_Clase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Gestión Riesgo Operacional - EVERGREEN**
En este espacio de trabajo se plantea como caso de estudio el análisis de los riesgos asociados a las operaciones del negocio de la Naviera EVERGREEN. Esta naviera Taiwanesa opera en 240 puertos alrededor del mundo utilizando 150 buques Canal de Panamá.
Los riesgos opercionales a los cuales esta expuesta la naviera están relacionados con la logistica portuaria frente a la carga y descarga de contenedores. De esta manera, este riesgo operacional se describe a través de dos variables aleatorias:
*  Frecuencia: Indica el número de contenedores que se quedan en tierra por semana (tiempo).
* Severidad (KUSD): Indica el costo de solucionar cada evento de de riesgo.
* Distribución Agregada de las Pérdidas: Producto entre la severidad y la frecuencia da como resultado la distribución agregada de las pérdidas (LDA - Loss Distribution Approach) y es precisamente que se hará la caracterización de esta distribución utilizando las medidas de tendencia de central (media, desviación) y las medidas de dispersión (coeficiente de asimetría, Kurtosis).


Para el enriquecimiento de los datos se utilizarán como referencia el Método Montecarlo (Ruleta Cargada - Muestreo Aleatorio), así como la Teoría de la Credibilidad, la cual muestra cuál es la afinidad que poseen los datos externos (Hapag Lloyd) frente a los datos que poseo internamente (EVERGREEN).

**0. Cargar las librerias de trabajo**


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns #Esta libreria para el trabajo estadístico
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**1. Se cargan los archivos de trabajo**

In [ ]:
#'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
#1. Aquí se carga la base de datos interna (Datos Observados - EVERGREEN)
#'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
nxl='/content/drive/MyDrive/University/Integración de Datos y Prospectiva/Clases/2/1. RiesgoOperacional_EVERGREEEN.xlsx'
XDB=pd.read_excel(nxl,sheet_name=0)
XDB.head(10)

#Procedemos con la construcción de la distribución agregada de las pérdidas
Freq=XDB.iloc[:,1]; Sev=XDB.iloc[:,2]; LDA=Freq*Sev

#'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
#2. Se procede con la carga de la base de datos (Datos Externos-HAPAG LLoyd)
#'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
nxle='/content/drive/MyDrive/University/Integración de Datos y Prospectiva/Clases/2/2. LógicaBorrosa_HAPAG_LLOYD.xlsx'
XDB2=pd.read_excel(nxle,sheet_name=0)
XDB2.head(10)

Freque=XDB2.iloc[:,3]; Sever=XDB2.iloc[:,4]; LDAe=Freque*Sever

#'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
#3. Comparación Estadística (Media, Varianza, Desviaciôn, Asimetrís, Kurtosis)
#'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
#3.1 Bases de Datos Observados (EVERGREEN)
uo,sigmao,Caso,Kuo,dfo=caracterizacion(LDA)
print("La media de los datos observados es:", uo)
print("El coeficiente de asimetría es:", Caso)

#3.2 Bases de Datos Externas (Hapag Lloyd)
ue,sigmae,Case,Kue,dfe=caracterizacion(LDAe)
print("La media de los datos externos es:", ue)
print("El coeficiente de asimetría es:", Case)

#''''''''''''''''''''''''''''''''''''''''''''
# 4. Teoría de Credibilidad
#''''''''''''''''''''''''''''''''''''''''''''
# 4.1 Media Hipótetica
NDo=len(LDA); NDe=len(LDAe) #Para saber la inercia de los datos
uhat=(NDo*uo+NDe*ue)/(NDo+NDe)

#4.2 EPV: Valor Esperado de la Varianza
EPV=(NDo*sigmao**2+NDe*sigmae**2)/(NDo+NDe) #Determinar la varianza conjunta de los datos

#4.3 VHM: Varianza Hipótetica de la Media
VHM=((NDo*uo**2+NDe*ue**2)/(NDo+NDe))-uhat**2

#4.4 Factor de Credibilidad
fc=(EPV/VHM)

#4.5 Credibilidad de la Base de Datos
Cr=1-(NDo/(NDo+fc)) #Significa
print("La credibilidad de la Base de Datos es:", Cr)

#'''''''''''''''''''''''''''''''''''''''''
#5. Parámetros de Riesgo
#'''''''''''''''''''''''''''''''''''''''''
#5.1 Pérdidas Asumibles (Media: Franquicia) : (Media Hipótetica de los Datos Integrados)
PEs=Cr*uo+(1-Cr)*ue
print("Se conoce como la franquicia de las pérdidas", PEs)

#Se procede con la estimación de las propiedades estadísticas
p1,p2,p3,p4,df=caracterizacion(LDA)

#Ahora se puede obtener las propiedades de la variable severidad
p1,p2,p3,p4,df=caracterizacion(Sev)


Se conoce como la franquicia de las pérdidas 281.0205616395309


**Análisis de credibilidad**

De acuerdo con las bases de datos disponibles para este estudio, podemos observar que las bases de datos a priori tuvieron media de datos similares (uo:279.65, ue: 287.42), esto se pudo evidenciar igualmente a través del coeficiente de asimetría, el cual indica una mayor acumulación de pérdidas menores en las operaciones de las navieras (Caso:3.79, Case:3.98). De acuerdo con la similitud de las métricas anteriores, el coeficiente de credibilidad que, evalua la calidad de los datos externos, nos indica que este alcanzo un valor por encima del 80% en promedio, lo que indica que más del 80% de los datos externos se pueden integrar con los datos internos

**2. Se procede con la clusterización estadística. La clusterización estadística esta definida por el Histograma de Frecuencias.**

In [ ]:
def caracterizacion(LDA):

  #Se procede con la creación de la distribución
  plt.figure(figsize=(10,5))
  sns.histplot(LDA,bins=10,kde=True)
  plt.xlabel("Pérdida (KUSD)")
  plt.grid()
  plt.show()

  #Se procede con la caracterización de cada una de las variables
  np.set_printoptions(suppress=True)
  NI=10    #Indica el número de clusters
  counts,bin_edges=np.histogram(LDA,bins=NI)
  print("El número de datos por intervalo es:")
  print(counts)
  print("Los intervalos inferiores:")
  print(bin_edges[:-1])
  print("Los intervalos superiores:")
  print(bin_edges[1:])
  XC=(bin_edges[:-1]+bin_edges[1:])/2

  #Se configura la tabla de los datos
  Tabla=np.column_stack((bin_edges[:-1],bin_edges[1:],XC,counts))
  df=pd.DataFrame(Tabla,columns=['LI','LS','XC','ND'])
  df.head(10)

  #Se procede con la estimación de la media
  fr=counts/np.sum(counts)
  u=np.sum(XC*fr)
  sigma2=np.sum(fr*(XC-u)**2)
  sigma=np.sqrt(sigma2)
  Cas=np.sum(fr*(XC-u)**3)/sigma**3
  Kur=(np.sum(fr*(XC-u)**4)/sigma**4)-3

  return u,sigma,Cas,Kur,df

**3. Se calculan las métricas estadísticas de tendencia central y dispersión: Media, Desviación, Asimetría y Kurtosis**

In [ ]:

u=np.mean(LDA)
var=np.var(LDA)
sigma=np.std(LDA) # Corrected: Calculate standard deviation of LDA

T2=np.column_stack((u,var,sigma))
df2=pd.DataFrame(T2,columns=['Media','Varianza','Desviacion'])
df2.head(1)

,Media,Varianza,Desviacion
0,231.266602,115581.511672,339.97281


In [ ]:
from scipy.stats import skew, kurtosis

# Calculate skewness
lda_skew = skew(LDA)

# Calculate kurtosis
lda_kurtosis = kurtosis(LDA)

# Optionally, add to the DataFrame with other metrics
df2_temp = df2.copy()
df2_temp['Asimetria'] = lda_skew
df2_temp['Kurtosis'] = lda_kurtosis

display(df2_temp)

Skewness (Asimetría) de LDA: 3.3891
Kurtosis de LDA: 14.5771


,Media,Varianza,Desviacion,Asimetria,Kurtosis
0,231.266602,115581.511672,339.97281,3.389058,14.577132


**Análisis de Resultados**

De las métricas de tendencia central y dispersión, podemos observar que los datos tienen la tendencia a ubicarse por debajo de la media como lo muestra el coeficiente de asimetría (3.3890), el cual esta por encima de la unidad, lo que indica que en general las pérdidas por dejar contenedores en tierra son pérdidas menores (pérdidas esperadas-no asegurables). Por su parte el coeficiente de curtosis, muestra que los datos se encuentran muy concentrados al lado izquierdo de la media de la distribución de las pérdidas, tal y como se puede observar en la gráfica del histograma de frecuencias. Esto refuerza la idea de menores pérdidas.